In [2]:
!pip install dill


     |████████████████████████████████| 95 kB 4.4 MB/s eta 0:00:011
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
import sys
sys.path.append('../')

import dill
from scripts.graph import Station, RouteStopArr, RouteStopDep, WalkingStop, Timetable

In [4]:
from os.path import join
import pandas as pd

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
DATA = '../data/'

# init stations

In [26]:
station_df = pd.read_csv(join(DATA, 'stations.csv'), index_col=0)
station_df.head()

,station_Id,STOP_NAME,STOP_LAT,STOP_LON,collect_list(end_route_stop_id)
0,8591097,"Zürich, Bristenstrasse",47.390079,8.487058,"['26-89-j20-1$Zürich, Bristenstrasse$D', '26-8..."
1,8590906,"Wangen b D'dorf, Dürrbach",47.408060,8.637589,"[""26-759-j20-1$Wangen b D'dorf, Dürrbach$D"", ""..."
2,8502559,"Waldegg, Birmensdorferstrasse",47.368303,8.463468,"['26-215-j20-1$Waldegg, Birmensdorferstrasse$A..."
3,8504801,"Hochfelden, Martinsmühle",47.523165,8.517367,"['26-535-j20-1$Hochfelden, Martinsmühle$D', '2..."
4,8503446,Zürich Landesmuseum (See),47.379046,8.541577,"['26-373-4-j20-1$Zürich Landesmuseum (See)$D',..."


In [29]:
stations = {}
for i, row in station_df.iterrows():
    stations[row['STOP_NAME']]= Station(int(row['station_Id']), row['STOP_NAME'], row['STOP_LAT'], row['STOP_LON'])

In [31]:
stations['Zürich, Bristenstrasse']

'Zürich, Bristenstrasse'

# init routestops

In [61]:
arrivals = pd.read_csv(join(DATA, 'arrivals_ext.csv'), index_col=0).fillna(30)
terminus = pd.read_csv(join(DATA, 'terminus_ext.csv'), index_col=0)
departure = pd.read_csv(join(DATA, 'departures.csv'), index_col=0)

In [62]:
arrivals.head(2)

,STOP_NAME,stop_sequence,end_route_stop_id,route_stop_id,travel_time,target_end_route_stop_id,route_id,route_name,transport_type
0,"Zürich, Hardplatz",16,"26-72-j20-1$Zürich, Hardplatz$A","26-72-j20-1$Zürich, Hardplatz",30.0,"26-72-j20-1$Zürich, Albisriederplatz$D",26-72-j20-1,Bus 72,Bus
1,"Zürich, Zwinglihaus",11,"26-72-j20-1$Zürich, Zwinglihaus$A","26-72-j20-1$Zürich, Zwinglihaus",60.0,"26-72-j20-1$Zürich, Bertastrasse$D",26-72-j20-1,Bus 72,Bus


In [63]:
terminus.head(1)

,STOP_NAME,stop_sequence,end_route_stop_id,route_stop_id,route_id,route_name,transport_type
0,"Zürich, Triemlispital",1,"26-80-j20-1$Zürich, Triemlispital$A","26-80-j20-1$Zürich, Triemlispital",26-80-j20-1,Bus 80,Bus


In [86]:
len(terminus), len(departure), len(arrivals)

(968, 10901, 21543)

In [79]:

departure.head(2).values

array([['26-8-j20-1$Zürich, Bernoulli-Häuser', '8591074', '26-8-j20-1',
        'Zürich, Bernoulli-Häuser',
        '26-8-j20-1$Zürich, Bernoulli-Häuser$D', 0.0, 'Tram 8', 'Tram'],
       ['26-8-j20-1$Zürich, Langmauerstrasse', '8591244', '26-8-j20-1',
        'Zürich, Langmauerstrasse',
        '26-8-j20-1$Zürich, Langmauerstrasse$D', 0.0, 'Tram 8', 'Tram']],
      dtype=object)

In [81]:
routestops_arr, routestops_dep = {}, {}
for i, row in terminus.iterrows():
    routestops_arr[row['end_route_stop_id']] = RouteStopArr(row['end_route_stop_id'],row['STOP_NAME'],
                                                            stations[row['STOP_NAME']], row['stop_sequence'], 
                                                            row['route_name'], row['transport_type'], 0, None)
    
for i, row in arrivals.iterrows():
    routestops_arr[row['end_route_stop_id']] = RouteStopArr(row['end_route_stop_id'],row['STOP_NAME'],
                                                            stations[row['STOP_NAME']], row['stop_sequence'], 
                                                            row['route_name'], row['transport_type'], 
                                                            row['travel_time'], None)
    
for i, row in departure.iterrows():
    routestops_dep[row['end_route_stop_id']] = RouteStopDep(row['end_route_stop_id'],row['STOP_NAME'],
                                                            stations[row['STOP_NAME']], -1, 
                                                            row['route_name'], row['transport_type'], 
                                                            row['travel_time'], None)


for i, row in arrivals.iterrows():
    routestops_arr[ row['end_route_stop_id']].set_prev_stop(routestops_dep[row['target_end_route_stop_id']])
    
for i, row in departure.iterrows():
    current_id = row['end_route_stop_id']
    previous_id = current_id[:-2] + '$A'
    routestops_dep[current_id].set_prev_stop(routestops_arr[previous_id])

KeyError: '69-3-Y-j20-1$Reppischhof$A'

In [84]:
arrivals.end_route_stop_id.str.startswith("69-3-Y-j20-1$Reppisc").sum()

0

# init timetable

In [7]:
# Dict[RouteStopDep, Tuple[List[int], List[Distrib]]]
pd.read_csv(join(DATA, 'timetable', 'timetable.csv'), nrows=1000)

,route_stop_id,arrival_time_complete_unix,departure_time_complete_unix,route_id,stop_id,trip_id,end,end_route_stop_id
0,"26-797-j20-1$Oberhasli, Moosacker",1.589388e+09,1.589388e+09,26-797-j20-1,8590731,104.TA.26-797-j20-1.4.R,D,"26-797-j20-1$Oberhasli, Moosacker$D"
1,"26-2-A-j20-1$Zürich Tiefenbrunnen, Bahnhof",1.589371e+09,1.589371e+09,26-2-A-j20-1,8576182,1096.TA.26-2-A-j20-1.6.H,A,"26-2-A-j20-1$Zürich Tiefenbrunnen, Bahnhof$A"
2,"26-80-j20-1$Zürich, Rautistrasse",1.589391e+09,1.589391e+09,26-80-j20-1,8591311,1096.TA.26-80-j20-1.11.R,D,"26-80-j20-1$Zürich, Rautistrasse$D"
3,"26-80-j20-1$Zürich, Zwielplatz",1.589392e+09,1.589392e+09,26-80-j20-1,8591446,1096.TA.26-80-j20-1.11.R,A,"26-80-j20-1$Zürich, Zwielplatz$A"
4,"26-11-j20-1$Zürich, Bad Allenmoos",1.589389e+09,1.589389e+09,26-11-j20-1,8591053,1103.TA.26-11-j20-1.21.R,D,"26-11-j20-1$Zürich, Bad Allenmoos$D"
...,...,...,...,...,...,...,...,...
995,"26-309-j20-1$Dietikon, Pestalozzi",1.589392e+09,1.589392e+09,26-309-j20-1,8590523,60.TA.26-309-j20-1.1.R,D,"26-309-j20-1$Dietikon, Pestalozzi$D"
996,"26-131-j20-1$Horgen, Bergli",1.589406e+09,1.589406e+09,26-131-j20-1,8573555,611.TA.26-131-j20-1.4.R,A,"26-131-j20-1$Horgen, Bergli$A"
997,"26-759-j20-1$Wallisellen, Herti",1.589391e+09,1.589391e+09,26-759-j20-1,8590889,622.TA.26-759-j20-1.5.R,A,"26-759-j20-1$Wallisellen, Herti$A"
998,"26-7-A-j20-1$Zürich, Mattenhof",1.589387e+09,1.589387e+09,26-7-A-j20-1,8591271,628.TA.26-7-A-j20-1.1.R,A,"26-7-A-j20-1$Zürich, Mattenhof$A"
